In [ ]:
!git clone https://github.com/lorenzonava96/TerraTrack.git
!pip uninstall earthengine-api -y
!pip uninstall geemap -y
!pip install -r TerraTrack/requirements.txt
# !pip install --upgrade earthengine-api geemap

In [ ]:
import rasterio
import os
import glob
import numpy as np
import pandas as pd
from skimage import exposure
from datetime import datetime
from scipy.ndimage import convolve
from scipy.signal import medfilt
from scipy.ndimage import gaussian_filter
import ee
import geemap
from TerraTrack.src import *

output_dir = 'outputs'

ee.Authenticate(auth_mode='notebook')


### **Define Area of Interest**

In [ ]:
# Create a Map instance
Map = geemap.Map()

# Add a satellite basemap for visualization (similar to Google Earth)
Map.add_basemap('HYBRID')

# Center the map globally
Map.setCenter(0, 0, 2)  # Center on the world with zoom level 2

# Display the map for user interaction
Map

### **Filter S2 Images**

In [ ]:
# Define filtering parameters
SUMMER_START = '-01-01'       # Start of seasonal filter (MM-DD)
SUMMER_END = '-12-30'         # End of seasonal filter (MM-DD)
START_YEAR = 2015             # First year to include
END_YEAR = 2025               # Last year to include
start_date = None     # Optional: remove images before this date ('YYYY-MM-DD')
final_date = None     # Optional: limit images up to this date ('YYYY-MM-DD')

# Image and mask filtering
CLOUD_COVER_MAX = 50          # Max allowable cloud cover per tile (%)
N_PER_YEAR = 5               # Max number of images sampled per year
mask_water = False            # Enable NDWI-based water masking
check_clouds = True           # Remove images with too much cloud in the ROI
cloud_threshold = 5           # Max allowable cloud cover in ROI (%)
check_snow = True            # Remove images with excessive snow in ROI
snow_threshold = 5            # Max allowable snow cover in ROI (%)

# Define region of interest from drawn features
roi = ee.FeatureCollection(Map.draw_features).geometry()

# Process Sentinel-2 imagery and extract terrain data
final_collection, morpho = process_sentinel2_data(
    roi, START_YEAR, END_YEAR, SUMMER_START, SUMMER_END,
    cloud_cover_max=CLOUD_COVER_MAX,
    n_per_year=N_PER_YEAR,
    mask_water=mask_water,
    check_clouds=check_clouds,
    cloud_threshold=cloud_threshold,
    check_snow=check_snow,
    snow_threshold=snow_threshold,
    start_date=start_date,
    final_date=final_date
)

# Print the size of the filtered image collection
print("Final collection size:", final_collection.size().getInfo())

# Print acquisition dates of selected images
dates = final_collection.aggregate_array('system:time_start') \
    .map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()

print("Acquisition dates in final_collection:")
for d in dates:
    print(d)

print("Processing completed. Final collection and morpho are ready to be downloaded.")

### **Downloading Processed Data**

In [ ]:
results = export_s2_composite_morpho_and_metadata(
    final_collection=final_collection,
    morpho=morpho,
    roi=roi,
    output_dir=output_dir
)


### **Processing Sentinel-2 Composite**

In [ ]:
process_composite_image(output_dir=output_dir, selection_method="auto")  #auto, manual

In [ ]:
composite_path = f"{output_dir}/S2_Composite_Filtered_8bit.tif"

orig, preprocessed_stack, zero_mask, orig_masked = load_and_preprocess_stack(
    composite_path=composite_path,
    preprocess_image_stack=preprocess_image_stack,
    method="cross_corr"
)

### **Defining Date Pairs for Analysis**

In [ ]:
min_separation = 1 # in years
max_separation = 5 # in years
reference_date = None # YYYY-MM-DD in case of a known constraint motion at a given time (e.g. earthquake)

dat1, dat2, separation, datax = define_date_pairs(f"{output_dir}/Updated_Metadata.csv",
                                                  min_separation=min_separation,
                                                  max_separation=max_separation,
                                                  reference_date=reference_date)

# **Feature Tracking Core**

### **Test on a Single Image Pair**

In [ ]:
import time

# --- Select Image Pair from Stack ---
# (Band indices: 0-based)
img1 = preprocessed_stack[:, :, 1]
img2 = preprocessed_stack[:, :, 20]

# --- Feature Tracking Parameters ---
method = 'block_matching'            # 'block_matching' or 'optical_flow'
match_func = 'fft_pcc'               # 'fft_ncc', 'fft_pcc', 'phase_cross_corr', or 'mean_optical_flow'
subpixel_method = 'parabolic'        # center_of_mass, parabolic, centroid, gaussian, os3, os5, os7, ipg, ensemble

block_size = 16
overlap = 0.8

# --- Displacement Limits ---
min_displacement = 0.1
max_displacement = 5

# --- Filtering Options ---
filter_params = {
    "apply_magnitude_filter": True,          # Remove displacements below/above thresholds
    "min_magnitude": min_displacement,                      # Minimum accepted motion
    "max_magnitude": max_displacement,       # Maximum accepted motion (usually block_size - 1)
    "apply_zero_mask_filter": False,         # Exclude pixels masked in any image (e.g., water/clouds)
    "apply_deviation_filter": False,         # Remove vectors far from statistical mean
    "std_factor": 2.5,                       # Standard deviation threshold for deviation filter
    "apply_remove_median_displacement": True,  # Subtract overall median motion (e.g., camera jitter)
    "apply_median_filter_step": False,       # Smooth vectors using a median filter
    "filter_size": 5,                        # Size of median filter kernel (if used)
    "apply_angular_coherence_filter": False, # Remove vectors that don’t align with dominant motion
    "angular_threshold": 50,                 # Max angular deviation (degrees)
    "smoothing_sigma": 1,                    # Angular coherence smoothing parameter
    "apply_erratic_displacement_filter": False,  # Remove isolated, noisy vectors
    "neighborhood_size": 20,                 # Radius for local filtering
    "deviation_threshold": 2.0,              # Threshold for local deviation
    "apply_pkr_filter": True,                # Remove low-quality matches using PKR
    "pkr_threshold": 1.3,                    # Minimum accepted peak-to-residual ratio
    "apply_snr_filter": True,                # Remove low-confidence vectors based on SNR
    "snr_threshold": 3,                      # Minimum signal-to-noise ratio
}

# --- Run Displacement Analysis ---
start_time = time.time()

u, v, feature_points, pkr, snr = displacement_analysis(
    img1=img1,
    img2=img2,
    method=method,
    block_size=block_size,
    overlap=overlap,
    match_func=match_func,
    subpixel_method=subpixel_method,
    zero_mask=zero_mask,
    filter_params=filter_params,
    plot=True,
    arrow_scale=0.1
)

full_feature_points = feature_points  # Store full-resolution grid

# --- Timing ---
elapsed_time = time.time() - start_time
print(f"BLOCK MATCHING — Process completed in {elapsed_time:.2f} seconds (1 pair).")


In [ ]:
# Estimate total processing time for all image pairs
num_pairs = len(separation)  # List of valid date pairs (from earlier step)
estimated_total_time = elapsed_time * num_pairs / 60  # in minutes

print(f"⏱️ Estimated total processing time for {num_pairs} pairs: {estimated_total_time:.1f} minutes (with parallel computing it'll be faster)")

### **Processing Image Pairs for Displacement Analysis**

In [ ]:
# FT parameters
method = 'block_matching' # block_matching, optical_flow
match_func = 'fft_pcc'      # 'fft_ncc', 'fft_pcc', 'phase_cross_corr', or 'mean_optical_flow'
subpixel_method='parabolic' # center_of_mass, parabolic, gaussian, os3, os5, os7, ipg, ensemble
block_size = 16
overlap = 0.8

# Filter parameters must be all off here
filter_params = {
    "apply_magnitude_filter": False,
    "min_magnitude": 0,
    "max_magnitude": max_displacement,
    "apply_zero_mask_filter": False,
    "apply_deviation_filter": False,
    "std_factor": 2.5,
    "apply_remove_median_displacement": False,
    "apply_median_filter_step": False,
    "filter_size": 5,
    "apply_angular_coherence_filter": False,
    "angular_threshold": 50,
    "smoothing_sigma": 1,
    "apply_erratic_displacement_filter": False,
    "neighborhood_size": 20,
    "deviation_threshold": 2.0,
    'apply_pkr_filter': False,
    'pkr_threshold': 1.3,
    'apply_snr_filter': False,
    'snr_threshold': 3,
}

# Run the processing function
results = process_image_pairs(
    dat1=dat1,
    dat2=dat2,
    datax=datax,
    preprocessed_stack=preprocessed_stack,
    zero_mask=zero_mask,
    filter_params=filter_params,
    method=method,
    block_size=block_size,
    overlap=overlap,
    match_func=match_func,
    subpixel_method=subpixel_method,
    max_workers=12,
    parallel=True)

print('FINALLY DONE !')

### **Saving Raw Feature Tracking (FT) Output**


In [ ]:
data = handle_predictions(
    output_dir=output_dir,
    method=method,
    match_func=match_func,
    results=results,
    separation=separation,
    orig=orig,
    dat1=dat1,
    dat2=dat2,
    save=True,
    load=True
)

# Access variables
all_u = data['all_u']
all_v = data['all_v']
all_feature_points = data['all_feature_points']
all_pkrs = data['all_pkrs']
all_snrs = data['all_snrs']
separation = data['separation']
study_area_image = data['study_area_image']
dat1 = data['dat1']
dat2 = data['dat2']

### **Pairwise Filtering**

In [ ]:
custom = {
        "apply_magnitude_filter": True,
        "min_magnitude": 0,
        "max_magnitude": block_size - 1,
        "apply_zero_mask_filter": False,
        "apply_deviation_filter": False,
        "std_factor": 2.5,
        "apply_remove_median_displacement": True,
        "apply_median_filter_step": False,
        "filter_size": 5,
        "apply_angular_coherence_filter": False,
        "angular_threshold": 50,
        "smoothing_sigma": 1,
        "apply_erratic_displacement_filter": False,
        "neighborhood_size": 20,
        "deviation_threshold": 2.0,
        "apply_pkr_filter": True,
        "pkr_threshold": 1.3,
        "apply_snr_filter": True,
        "snr_threshold": 3,
}

filtered_all_u, filtered_all_v, filtered_all_feature_points, used_params = filter_all_pairs(
    all_u, all_v, all_feature_points, all_pkrs, all_snrs,
    filter_displacements=filter_displacements,
    block_size=block_size,
    zero_mask=zero_mask,
    filter_params=custom,
    strict_quality_alignment=False,
)


### **Median Displacement Computation**  

In [ ]:
pixel_size = 10   # Pixel resolution in meters
apply_slope_correction=False # Correct estimates according to slope angle

# Resample morpho image to match `study_area_image` shape
morpho_path = f"{output_dir}/morpho.tif"
output_path = f"{output_dir}/resampled_morpho.tif"
resampled_morpho_path, resampled_transform = resample_morpho_to_match(study_area_image.shape, morpho_path, output_path)

# Open and plot the resampled DEM, slope, and aspect
with rasterio.open(resampled_morpho_path) as src:
    resampled_dem = src.read(1)  # Band 1: Resampled DEM
    resampled_slope = src.read(2)  # Band 2: Resampled slope
    resampled_aspect = src.read(3)  # Band 3: Resampled aspect

# Step 1: Accumulate Displacements
displacement_data = accumulate_displacement(filtered_all_u, filtered_all_v, filtered_all_feature_points, separation)

# Step 2: Calculate Median Displacement
median_feature_points, median_u, median_v, median_magnitude, median_angles = calculate_median_displacement(displacement_data, pixel_size, slope_map=resampled_slope, apply_slope_correction=apply_slope_correction)

plot_displacement_field(median_feature_points, median_u, median_v, median_magnitude, study_area_image, arrow_scale=0.1)

### **Final Displacement Filtering & Visualization**  

In [ ]:
# Specify which filters to use & define parameters
manual_threshold=0.2                   # If None, 95th percentile is used

use_angular_coherence = True            # Use angular coherence filter
angular_threshold_degrees = 30          # Angular threshold in degrees
use_slope_filter = True                 # Skip slope filter
min_slope_threshold = 5                 # Minimum slope in degrees
use_aspect_filter = True                # Use aspect filter
aspect_tolerance = 90                   # Allowable deviation from downslope direction in degrees
use_clustering = True                   # Skip clustering filter
clustering_params = (20, 9)             # Clustering parameters: (eps, min_samples)

arrow_scale = 0.3                       # Scale of the vector arrow

fil_median_feature_points, fil_median_u, fil_median_v, fil_median_magnitude = filter_final_map(
    median_feature_points, median_u, median_v, median_magnitude, median_angles,
    resampled_slope, resampled_aspect, study_area_image,
    angular_threshold_degrees=angular_threshold_degrees, min_slope_threshold=min_slope_threshold,
    aspect_tolerance=aspect_tolerance, smoothing_sigma=1, clustering_params=clustering_params,
    use_angular_coherence=use_angular_coherence, use_slope_filter=use_slope_filter,
    use_aspect_filter=use_aspect_filter, use_clustering=use_clustering, arrow_scale=arrow_scale,
    manual_threshold=manual_threshold
)

### **Generating & Saving Motion Colormap**  

In [ ]:
# From cartesian to raster

# filtered map
fil_u_map, fil_v_map, fil_magnitude_map, fil_angle_map = create_raster_maps(
    fil_median_feature_points, fil_median_u, fil_median_v, study_area_image, block_size, overlap
)

# full map
u_map, v_map, magnitude_map, angle_map = create_raster_maps(
    median_feature_points, median_u, median_v, study_area_image, block_size, overlap
)

processed_mask = process_mask(fil_magnitude_map)

masked_magnitude = np.where(processed_mask, magnitude_map, np.nan)

orig_path = f'{output_dir}/S2_Composite_Filtered_8bit.tif'
output_path = f"{output_dir}/output_motion_colormap.tif"

# Generate the magnitude map
overlay_magnitude_map(orig[..., 0], masked_magnitude, block_size=block_size, overlap=overlap)

save_as_geotiff(orig_path, output_path, masked_magnitude, block_size, overlap)

### **Time Series Reconstruction & Velocity Estimation**

In [ ]:
cartesian_points = get_cartesian_points_from_mask(processed_mask, study_area_image.shape[:2])

# Convert each row to a tuple for set operations.
set_cartesian = set(map(tuple, cartesian_points))
set_fil = set(map(tuple, median_feature_points))

# Get the intersection (points that appear in both).
common_points_set = set_cartesian.intersection(set_fil)

# Convert back to a NumPy array.
ts_points = np.array(list(common_points_set))

displacement_data = accumulate_displacement_with_placeholders(
    filtered_all_u, filtered_all_v, filtered_all_feature_points, separation, ts_points, dat1, dat2, pixel_size, all_pkrs, all_snrs
)


Reconstructs pixel-wise displacement time series and estimates average velocity from pairwise results.

🧠 **Tip**: Use `'weighted'` for better handling of irregular observation frequency and variable data quality.

In [ ]:
method = 'lsqr' # 'weighted' or 'midpoint' or 'lsqr'
months_per_bin = 6
min_snr=3
min_pkr=1.3

# the following are just for lsqr
time_step='3M' # Solve for velocity every n months
weight_by='snr' # Weight by snr or pkr ratio

velocity_estimates = estimate_velocity_time_series(
    displacement_data,
    method=method,
    months_per_bin=months_per_bin,
    min_snr=min_snr,
    min_pkr=min_pkr,
    time_step=time_step,
    weight_by=weight_by,
    velocity_units='m/year'
)

Create Multiband .tif (creates 1 georeferenced map for each time step in the time series)

Create a .gif time lapse of the same

In [ ]:
tif_path = f"{output_dir}/{output_dir}_magnitude_multiband.tif"
output_gif = f"{output_dir}/{output_dir}_magnitude_multiband.gif"

create_multiband_magnitude_tif(velocity_estimates, study_area_image, output_dir, block_size, overlap, output_filename=f"{output_dir}_magnitude_multiband.tif")

# study_area_image should be loaded as a NumPy array and have the same resolution as your composite.
create_gif_with_background_and_colorbar(tif_path, study_area_image, output_gif, duration=1000.0, cmap="viridis", alpha=0.6, velocity_estimates=velocity_estimates)

Save time series WE, NS and Magnitude in files compatible with InSAR Explorer

In [ ]:
# Create velocity time series CSVs with median velocities for EW and SN components
csv_data_we, csv_data_ns, csv_data_mag = prepare_csv_with_components(velocity_estimates, geotiff_path=f'{output_dir}/S2_Composite_Filtered_8bit.tif')

# Save to CSV (optional)
csv_data_we.to_csv(f'{output_dir}/velocity_time_series_we.csv', index=False)
csv_data_ns.to_csv(f'{output_dir}/velocity_time_series_ns.csv', index=False)
csv_data_mag.to_csv(f'{output_dir}/velocity_time_series_mag.csv', index=False)

In [ ]:
# Plot fastes points
plot_fastest_points_components(csv_data_we, csv_data_ns, top_n=5)

In [ ]:
import matplotlib.pyplot as plt
import rasterio
from rasterio.warp import transform_bounds
import pandas as pd

# Paths to input files
geotiff_path = f'{output_dir}/S2_Composite_Filtered_8bit.tif'
csv_ew_path = f'{output_dir}/velocity_time_series_we.csv'
csv_sn_path = f'{output_dir}/velocity_time_series_ns.csv'

# Load CSVs
csv_data_ew = pd.read_csv(csv_ew_path)
csv_data_sn = pd.read_csv(csv_sn_path)

# Combine WE and NS velocities into a single DataFrame
csv_data_ew['mean_velocity'] = (csv_data_ew['median_velocity']**2 + csv_data_sn['median_velocity']**2)**0.5

# Load GeoTIFF image and get transform & CRS
with rasterio.open(geotiff_path) as src:
    image = src.read(1)  # Read the first band
    transform = src.transform
    src_crs = src.crs

    # Get image bounds in the native CRS
    left = transform[2]
    top = transform[5]
    right = left + transform[0] * image.shape[1]
    bottom = top + transform[4] * image.shape[0]

    # Transform bounds to EPSG:84 (WGS84) so they match the CSV lat/lon
    bounds_84 = transform_bounds(src_crs, 'EPSG:4326', left, bottom, right, top)

# bounds_84 is in the order (min_lon, min_lat, max_lon, max_lat)
xmin, ymin, xmax, ymax = bounds_84

# Plot the image using the transformed bounds
plt.figure(figsize=(12, 8))
plt.imshow(image, cmap='gray', extent=[xmin, xmax, ymin, ymax])
plt.title('Mean Velocity from Time Series Overlaid on Image')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Overlay mean velocity as scatter points (CSV coordinates assumed to be in EPSG:84)
scatter = plt.scatter(
    csv_data_ew['longitude'], csv_data_ew['latitude'],
    c=csv_data_ew['mean_velocity'], cmap='coolwarm', s=15, edgecolor='k', alpha=0.7
)
plt.colorbar(scatter, label='Mean Velocity (m/year)')
plt.grid()
plt.tight_layout()
plt.show()

### ⬇️ Downloading Outputs

To download files directly from Colab to your computer. This creates a zip of the entire outputs folder and downloads it to device.


In [ ]:
import shutil
from google.colab import files

# Zip the folder
shutil.make_archive(f'Terratrack_Results_{output_dir}', 'zip', 'outputs')

# Download
files.download(f'Terratrack_Results_{output_dir}.zip')

In [ ]:
# Run this after the cells above to set an alert sound when the processing is finished
play_alert()

# Inverse Velocity

In [ ]:
component = 'mag' # 'we', 'ns', 'mag'
failure_dates_list = compute_inverse_velocity_failure_dates(f"Chaos/velocity_time_series_{component}.csv", n_points_for_fit_list=[4,5,6,7])
failure_counts = failure_date_statistics(failure_dates_list)
plot_failure_distribution(failure_counts)